# Bermuda Concepts

This notebook introduces the core concepts of the Bermuda library and demonstrates its fundamental capabilities for working with insurance loss triangles.

## Design Rationale

Bermuda is designed with several key principles in mind:

1. **Tabular Triangle Structure**: Unlike traditional triangle representations, Bermuda uses tabular structures that can include multiple data types in each evaluation date (paid loss, earned premium, claim counts, etc.)

2. **Immutability**: Triangle objects are immutable - any modification creates a new triangle, ensuring data integrity

3. **Metadata Flow**: Metadata flows from cells to triangles, allowing rich data annotation and automatic consolidation

4. **Flexible Data Handling**: Support for both cumulative and incremental views, missing data, and multi-slice triangles

More details at: https://ledger-investing-bermuda-ledger.readthedocs-hosted.com/en/latest/user-guide/design.html

## Setup and Data Loading

In [ ]:
import bermuda as tri
import altair as alt
from datetime import date
import numpy as np

# Enable HTML rendering for charts
alt.renderers.enable("html")

In [ ]:
# Load the built-in Meyers triangle data
triangle = tri.meyers_tri
print(triangle)

## Cells and Triangles

### Understanding Cells

Cells are the fundamental building blocks of triangles. Each cell represents a single evaluation of losses for a specific experience period.

In [ ]:
# Examine a single cell
first_cell = triangle[0]
print("First cell in triangle:")
print(first_cell)
print()
print("Cell components:")
print(f"  Period: {first_cell.period_start} to {first_cell.period_end}")
print(f"  Evaluation date: {first_cell.evaluation_date}")
print(f"  Values: {first_cell.values}")
print(f"  Metadata: {first_cell.metadata}")

### Building Up to Triangles

Triangles are collections of cells organized by experience periods and evaluation dates. Metadata flows through cells to consolidate at the triangle level.

In [ ]:
# Demonstrate metadata consolidation
print("Triangle-level metadata consolidation:")
print(f"  Number of cells: {len(triangle)}")
print(f"  Fields present: {triangle.fields}")
print(f"  Common metadata: {triangle.common_metadata}")

In [ ]:
# Demonstrate what happens when data is missing
# Create a triangle with missing earned_premium in some cells
cells_with_missing = []
for i, cell in enumerate(triangle):
    if i % 5 == 0:  # Remove earned_premium from every 5th cell
        new_values = {k: v for k, v in cell.values.items() if k != 'earned_premium'}
        new_cell = tri.CumulativeCell(
            period_start=cell.period_start,
            period_end=cell.period_end,
            evaluation_date=cell.evaluation_date,
            values=new_values,
            metadata=cell.metadata
        )
        cells_with_missing.append(new_cell)
    else:
        cells_with_missing.append(cell)

triangle_with_missing = tri.Triangle(cells_with_missing)
print("Triangle with missing data:")
print(triangle_with_missing)
print("\nNotice 'earned_premium' now shows as optional with coverage percentage")

### Demonstrating Immutability

Triangles are immutable - modifications create new triangle objects

In [ ]:
# Original triangle remains unchanged
print(f"Original triangle ID: {id(triangle)}")
print(f"Original has {len(triangle)} cells")

# Create a filtered triangle
filtered = triangle.filter(lambda cell: cell.period_start.year >= 1990)
print(f"\nFiltered triangle ID: {id(filtered)}")
print(f"Filtered has {len(filtered)} cells")

print("\nOriginal triangle is unchanged - immutability preserved!")

## Triangle Components

### Experience Range and Resolution

In [ ]:
# Examine experience periods
print("Experience range and resolution:")
print(f"  Range: {triangle.periods[0][0]} to {triangle.periods[-1][1]}")
print(f"  Resolution: {triangle.experience_resolution} months")
print(f"  Number of periods: {len(triangle.periods)}")

print("\nFirst 3 periods:")
for i, period in enumerate(triangle.periods[:3]):
    print(f"  {i+1}. {period[0]} to {period[1]}")

In [ ]:
# Examine evaluation dates
print("Evaluation range and resolution:")
print(f"  Range: {triangle.evaluation_dates[0]} to {triangle.evaluation_dates[-1]}")
print(f"  Resolution: {triangle.evaluation_resolution} months")
print(f"  Number of evaluation dates: {len(triangle.evaluation_dates)}")

### Development Lag - Ledger's Convention

**Important**: Ledger interprets dev lag to be the number of months away from period end. Dev lag 0 corresponds to the evaluation date being at period_end.

In [ ]:
# Demonstrate dev lag convention
print("Ledger's Dev Lag Convention:")
print("Dev lag 0 = evaluation at period_end\n")

# Find cells with dev lag 0
for cell in triangle[:10]:  # Check first 10 cells
    if cell.evaluation_date == cell.period_end:
        dev_lag = tri.calculate_dev_lag(cell.period_end, cell.evaluation_date)
        print(f"Cell with dev lag 0:")
        print(f"  Period: {cell.period_start} to {cell.period_end}")
        print(f"  Evaluation: {cell.evaluation_date}")
        print(f"  Dev lag: {dev_lag} months")
        break

### Square Triangles

A "square" triangle has the same length of time for evaluation dates as for development lag.

In [ ]:
# Check if triangle is square
max_dev_lag = max(tri.calculate_dev_lag(cell.period_end, cell.evaluation_date) 
                  for cell in triangle)
experience_months = (triangle.periods[-1][1] - triangle.periods[0][0]).days / 30.44

print(f"Maximum development lag: {max_dev_lag} months")
print(f"Experience period span: ~{experience_months:.0f} months")
print(f"Is approximately square: {abs(max_dev_lag - experience_months) < 12}")

### Cumulative vs Incremental Views

In [ ]:
# Find a cell with multiple evaluations for the same period
target_period = triangle.periods[0]  # First accident period

# Get cells for this period
period_cells = [cell for cell in triangle 
                if cell.period_start == target_period[0] 
                and cell.period_end == target_period[1]]
period_cells.sort(key=lambda x: x.evaluation_date)

# Show the last cell as cumulative
if len(period_cells) >= 2:
    last_cell = period_cells[-1]
    print("Cumulative cell (last evaluation):")
    print(f"  Period: {last_cell.period_start} to {last_cell.period_end}")
    print(f"  Evaluation: {last_cell.evaluation_date}")
    print(f"  Cumulative paid loss: ${last_cell.values['paid_loss']:,.0f}")
    
    # Convert to incremental
    incremental_tri = triangle.to_incremental()
    
    # Find the same cell in incremental view
    for inc_cell in incremental_tri:
        if (inc_cell.period_start == last_cell.period_start and 
            inc_cell.evaluation_date == last_cell.evaluation_date):
            print("\nIncremental cell (same position):")
            print(f"  Previous evaluation: {inc_cell.prev_evaluation_date}")
            print(f"  Incremental paid loss: ${inc_cell.values['paid_loss']:,.0f}")
            break

## Clipping and Filtering

Bermuda provides powerful methods for subsetting triangles with method chaining support.

In [ ]:
# Clipping to limit years
clipped = triangle.clip(
    min_period=date(1990, 1, 1),
    max_period=date(1995, 12, 31)
)

print("Original triangle:")
print(f"  Periods: {triangle.periods[0][0]} to {triangle.periods[-1][1]}")
print(f"  Cells: {len(triangle)}")

print("\nClipped triangle (1990-1995):")
print(f"  Periods: {clipped.periods[0][0]} to {clipped.periods[-1][1]}")
print(f"  Cells: {len(clipped)}")

In [ ]:
# Visualize the clipped data
clipped.plot_data_completeness()

In [ ]:
# Limit evaluation dates
eval_limited = clipped.clip(max_eval=date(1996, 12, 31))

print("After limiting evaluation dates:")
print(f"  Evaluation range: {eval_limited.evaluation_dates[0]} to {eval_limited.evaluation_dates[-1]}")
print(f"  Cells: {len(eval_limited)}")

# Show completeness
eval_limited.plot_data_completeness()

In [ ]:
# Filtering - keep only even years using method chaining
even_years = (triangle
              .filter(lambda cell: cell.period_start.year % 2 == 0)
              .clip(max_eval=date(2000, 12, 31)))

print("Filtered to even years with method chaining:")
print(f"  Cells: {len(even_years)}")
print("  Periods included:")
for period in even_years.periods[:5]:
    print(f"    {period[0].year}")

## Date Utilities

Bermuda provides helpful date manipulation functions.

In [ ]:
# add_months utility
start_date = date(2020, 1, 31)
print(f"Starting date: {start_date}")
print("\nAdding months:")
for months in [1, 3, 6, 12, 24]:
    new_date = tri.add_months(start_date, months)
    print(f"  +{months} months: {new_date}")

In [ ]:
# calculate_dev_lag utility
period_end = date(2020, 12, 31)
evaluations = [
    date(2020, 12, 31),  # Dev lag 0
    date(2021, 12, 31),  # Dev lag 12
    date(2022, 6, 30),   # Dev lag 18
    date(2023, 12, 31),  # Dev lag 36
]

print(f"Period end: {period_end}")
print("\nDevelopment lags:")
for eval_date in evaluations:
    dev_lag = tri.calculate_dev_lag(period_end, eval_date)
    print(f"  Evaluation {eval_date}: {dev_lag} months")

## Visualization

Let's explore some key visualizations available in Bermuda.

In [ ]:
# Right edge plot - shows latest evaluation for each period
triangle.plot_right_edge()

In [ ]:
# Use a smaller subset for mountain and ballistic plots
viz_triangle = triangle.clip(
    min_period=date(1990, 1, 1),
    max_period=date(1993, 12, 31),
    max_eval=date(1998, 12, 31)
)

print(f"Subset for visualization: {len(viz_triangle)} cells")

In [ ]:
# Mountain plot - shows development patterns
viz_triangle.plot_mountain()

In [ ]:
# Mountain plot for reported loss
viz_triangle.plot_mountain('Reported Loss')

In [ ]:
# Ballistic plot - shows trajectories
viz_triangle.plot_ballistic()

## Summary

In this notebook, we've covered the fundamental concepts of Bermuda:

1. **Design Philosophy**: Tabular triangles with metadata flow and immutability
2. **Cells and Triangles**: Building blocks and their relationship
3. **Triangle Components**: Periods, evaluation dates, and resolutions
4. **Development Lag Convention**: Ledger's interpretation where dev lag 0 = period end
5. **Square Triangles**: Equal time spans for experience and development
6. **Cumulative vs Incremental**: Different views of the same data
7. **Clipping and Filtering**: Powerful subsetting with method chaining
8. **Date Utilities**: Helper functions for date manipulation
9. **Visualizations**: Built-in charts for triangle analysis

These concepts form the foundation for working with insurance loss triangles in Bermuda.